In [1]:
#import splinter and beautiful soup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd



In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\prisc\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


In [3]:
#visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
#optional delay for loading page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Scientists Explore Outback as Testbed for Mars </div>

In [6]:
#use the parent element to find the first ' a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Scientists Explore Outback as Testbed for Mars '

In [7]:
#use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars."

### Featured Images

In [8]:
#visit URL
url= 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
#find and click the full imagine button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
#parse the resulting html with soup
html= browser.html
img_soup = soup(html, 'html.parser')

In [11]:
#find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [12]:
#use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [15]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)
html = browser.html
hemi_soup = soup(html,'html.parser')
hemi_soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [17]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
# 3. Write code to retrieve the image urls and titles for each hemisphere.
# Parse the html with soup
html = browser.html
hemisphere_soup = soup(html, 'html.parser')

# Find the number of pictures to scan
pics_count = len(hemisphere_soup.select("div.item"))

# for loop over the link of each sample picture
for i in range(pics_count):
    # Create an empty dict to hold the search results
    results = {}
    # Find link to picture and open it
    link_image = hemisphere_soup.select("div.description a")[i].get('href')
    browser.visit(f'https://marshemispheres.com/{link_image}')
    
    # Parse the new html page with soup
    html = browser.html
    image_soup = soup(html, 'html.parser')
    # Get the full image link
    img_url = image_soup.select_one("div.downloads ul li a").get('href')
    # Get the full image title
    img_title = image_soup.select_one("h2.title").get_text()
    # Add extracts to the results dict
    results = {
        'img_url': f'https://marshemispheres.com/{img_url}',
        'title': img_title }
    
    # Append results dict to hemisphere image urls list
    hemisphere_image_urls.append(results)
    
    # Return to main page
    browser.back()


In [18]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
# 5. Quit the browser
browser.quit()